In [1]:
pip install transformers

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the pre-trained GPT-2 language model
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
poem_text = """One must have a mind of winter
To regard the frost and the boughs
Of the pine-trees crusted with snow;
And have been cold a long time
To behold the junipers shagged with ice,
The spruces rough in the distant glitter
Of the January sun; and not to think
Of any misery in the sound of the wind,
In the sound of a few leaves,
Which is the sound of the land
Full of the same wind
That is blowing in the same bare place
For the listener, who listens in the snow,
And, nothing himself, beholds
Nothing that is not there and the nothing that is."""


In [8]:
poem_lines = poem_text.strip().split('\n')
print(f"Number of lines in the poem: {len(poem_lines)}")
print("First 3 lines:")
for i, line in enumerate(poem_lines[:3]):
    print(f"Line {i+1}: {line}")

Number of lines in the poem: 15
First 3 lines:
Line 1: One must have a mind of winter
Line 2: To regard the frost and the boughs
Line 3: Of the pine-trees crusted with snow;


In [9]:
reconstructed_lines = []

In [10]:
poem_lines = poem_text.strip().split('\n')
print(f"Number of lines in the poem: {len(poem_lines)}")
print("First 3 lines:")
for i, line in enumerate(poem_lines[:3]):
    print(f"Line {i+1}: {line}")

Number of lines in the poem: 15
First 3 lines:
Line 1: One must have a mind of winter
Line 2: To regard the frost and the boughs
Line 3: Of the pine-trees crusted with snow;


In [40]:
reconstructed_lines = []
target_prediction_index = 35

In [41]:
import torch
import re

for i, line in enumerate(poem_lines):
    # Split line into words and clean each word from special characters
    cleaned_line_words = [re.sub(r'[^À-ſ\w]', '', word) for word in line.split()]
    # Filter out empty strings that might result from cleaning (e.g., '...')
    words = [word for word in cleaned_line_words if word]

    if not words:
        reconstructed_lines.append('')
        continue

    last_word = words[-1]
    prefix = ' '.join(words[:-1])

    # Encode the prefix
    input_ids = tokenizer.encode(prefix, return_tensors='pt')

    # Get model predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # Get the probabilities for the next token
    next_token_logits = predictions[0, -1, :]
    # Sort by probability in descending order and get token IDs
    sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)

    # Get the token at the target prediction index
    if len(sorted_indices) > target_prediction_index:
        predicted_token_id = sorted_indices[target_prediction_index].item()
        predicted_word = tokenizer.decode([predicted_token_id]).strip()
    else:
        # Fallback if fewer predictions are available than the target index
        predicted_word = "<UNKNOWN>"
        if len(sorted_indices) > 0:
            predicted_token_id = sorted_indices[0].item()
            predicted_word = tokenizer.decode([predicted_token_id]).strip()

    # Reconstruct the line with the new word
    reconstructed_line = f"{prefix} {predicted_word}".strip()
    reconstructed_lines.append(reconstructed_line)

print("Finished processing all lines with the new poem.")

Finished processing all lines with the new poem.


In [42]:
print("Reconstructed Poem :")
print("--------------------------------------------------")
print('\n'.join(reconstructed_lines))

Reconstructed Poem :
--------------------------------------------------
One must have a mind of justice
To regard the frost and the ensuing
Of the pinetrees crusted with her
And have been cold a long amount
To behold the junipers shagged with blue
The spruces rough in the distant stars
Of the January sun and not to begin
Of any misery in the sound of the man
In the sound of a few s
Which is the sound of the police
Full of the same things
That is blowing in the same bare throat
For the listener who listens in the box
And nothing himself really
Nothing that is not there and the nothing that may
